# Protein: Nightingale

##  Post-Unblinding Re-Analysis

In [20]:
import pandas as pd

# File paths
post_unblinding = r"E:\ADNI\AD_CN\protein\ADNINIGHTINGALE2_23Jun2025.csv"
img_acquisitions_info = r"E:\ADNI\AD_CN\AD,CN_Acquisitions_25Jun2025.csv"

# 1. Load and deduplicate protein data by RID (baseline t=0)
protein_df = pd.read_csv(post_unblinding)
protein_df = protein_df.drop_duplicates(subset='RID')

# 2. Load MRI acquisition info and filter for baseline (t=0)
acq_df = pd.read_csv(img_acquisitions_info)
acq_bl = acq_df[acq_df['acquisition_visit'].str.lower() == 'bl'].copy()

# 3. Extract numeric RID from subject_id for matching
acq_bl['RID'] = acq_bl['subject_id'].str.split('_').str[-1].astype(int)

# 4. Merge protein and MRI info on RID
merged_df = pd.merge(protein_df, acq_bl[['RID', 'MRI_acquired']], on='RID', how='left')

# 5. Summary statistics
n_protein_baseline = len(protein_df)
n_protein_with_mri = merged_df['MRI_acquired'].eq(1).sum()

print(f"Number of protein baseline (t=0) samples: {n_protein_baseline}")
print(f"Number of protein baseline samples with MRI baseline data: {n_protein_with_mri}")

# 6. Save merged dataframe
merged_df.to_csv("protein/Nightingale/protein_mri_baseline_merged.csv", index=False)

Number of protein baseline (t=0) samples: 1646
Number of protein baseline samples with MRI baseline data: 102


## Original NMR Analysis

In [30]:
import pandas as pd

# File paths
protein = r"E:\ADNI\AD_CN\protein\ADNINIGHTINGALE_20210219_23Jun2025.csv"
img_acquisitions_info = r"E:\ADNI\AD_CN\AD,CN_Acquisitions_25Jun2025.csv"
output_path = r"E:\ADNI\AD_CN\protein\Nightingale\ADNINIGHTINGALE_20210219_23Jun2025_merged.csv"

# 1. Load protein data
protein_df = pd.read_csv(protein)

# 2. Remove rows with 'NA' in VISCODE2
protein_df = protein_df[protein_df['VISCODE2'].str.lower() != 'na']

# 3. Load MRI acquisition info
acq_df = pd.read_csv(img_acquisitions_info)

# 4. Extract numeric rid from subject_id for matching
acq_df['rid'] = acq_df['subject_id'].str.split('_').str[-1].astype(int)

# 5. Get unique timestamps from protein data
unique_timestamps = protein_df['VISCODE2'].str.lower().unique()

# 6. Initialize summary statistics
summary_stats = {}

# 7. Process each timestamp
for timestamp in unique_timestamps:
    # Filter protein data for current timestamp
    protein_timestamp = protein_df[protein_df['VISCODE2'].str.lower() == timestamp].copy()
    
    # Filter MRI acquisition info for current timestamp
    acq_timestamp = acq_df[acq_df['acquisition_visit'].str.lower() == timestamp].copy()
    
    # Count unique rids for protein data
    n_protein = protein_timestamp['RID'].nunique()
    
    # Merge protein and MRI info
    merged_timestamp = pd.merge(protein_timestamp, acq_timestamp[['rid', 'MRI_acquired']], left_on='RID', right_on='rid', how='left', suffixes=('', '_MRI'))
    
    # Count protein data with MRI
    n_protein_with_mri = merged_timestamp['MRI_acquired'].eq(1).sum()
    
    # Store summary statistics
    summary_stats[timestamp] = {
        'n_protein': n_protein,
        'n_protein_with_mri': n_protein_with_mri
    }
    
    # Print summary for current timestamp
    print(f"Number of protein {timestamp} (t={list(unique_timestamps).index(timestamp)}): {n_protein}")
    print(f"Number of protein {timestamp} with MRI {timestamp} data: {n_protein_with_mri}")

# 8. Combine all merged dataframes
merged_dfs = []
for timestamp in unique_timestamps:
    protein_timestamp = protein_df[protein_df['VISCODE2'].str.lower() == timestamp].copy()
    acq_timestamp = acq_df[acq_df['acquisition_visit'].str.lower() == timestamp].copy()
    merged_timestamp = pd.merge(protein_timestamp, acq_timestamp[['rid', 'MRI_acquired']], left_on='RID', right_on='rid', how='left', suffixes=('', '_MRI'))
    merged_dfs.append(merged_timestamp)

merged_df = pd.concat(merged_dfs, ignore_index=True)

# 9. Save merged dataframe
merged_df.to_csv(output_path, index=False)

print(f"\nMerged DataFrame saved to: {output_path}")

Number of protein bl (t=0): 1643
Number of protein bl with MRI bl data: 106
Number of protein sc (t=1): 1
Number of protein sc with MRI sc data: 0
Number of protein f (t=2): 1
Number of protein f with MRI f data: 0
Number of protein nv (t=3): 1
Number of protein nv with MRI nv data: 0

Merged DataFrame saved to: E:\ADNI\AD_CN\protein\Nightingale\ADNINIGHTINGALE_20210219_23Jun2025_merged.csv


# Protein: Hu Lab

In [25]:
import pandas as pd

# File paths
protein = r"E:\ADNI\AD_CN\protein\Hu Lab\ADNI_HULAB_23Jun2025.csv"
img_acquisitions_info = r"E:\ADNI\AD_CN\AD,CN_Acquisitions_25Jun2025.csv"
output_path = r"E:\ADNI\AD_CN\protein\Hu Lab\protein_mri_merged.csv"

# 1. Load protein data
protein_df = pd.read_csv(protein)

# 2. Load MRI acquisition info
acq_df = pd.read_csv(img_acquisitions_info)

# 3. Extract numeric RID from subject_id for matching
acq_df['RID'] = acq_df['subject_id'].str.split('_').str[-1].astype(int)

# 4. Define timestamps
timestamps = ['bl', 'm12', 'm24', 'm36', 'm48', 'm60']

# 5. Initialize summary statistics
summary_stats = {}

# 6. Process each timestamp
for timestamp in timestamps:
    # Filter protein data for current timestamp
    protein_timestamp = protein_df[protein_df['VISCODE'].str.lower() == timestamp].copy()
    
    # Filter MRI acquisition info for current timestamp
    acq_timestamp = acq_df[acq_df['acquisition_visit'].str.lower() == timestamp].copy()
    
    # Count unique RIDs for protein data
    n_protein = protein_timestamp['RID'].nunique()
    
    # Merge protein and MRI info
    merged_timestamp = pd.merge(protein_timestamp, acq_timestamp[['RID', 'MRI_acquired']], on='RID', how='left', suffixes=('', '_MRI'))
    
    # Count protein data with MRI
    n_protein_with_mri = merged_timestamp['MRI_acquired'].eq(1).sum()
    
    # Store summary statistics
    summary_stats[timestamp] = {
        'n_protein': n_protein,
        'n_protein_with_mri': n_protein_with_mri
    }
    
    # Print summary for current timestamp
    print(f"Number of protein {timestamp} (t={timestamps.index(timestamp)}): {n_protein}")
    print(f"Number of protein {timestamp} with MRI {timestamp} data: {n_protein_with_mri}")

# 7. Combine all merged dataframes
merged_dfs = []
for timestamp in timestamps:
    protein_timestamp = protein_df[protein_df['VISCODE'].str.lower() == timestamp].copy()
    acq_timestamp = acq_df[acq_df['acquisition_visit'].str.lower() == timestamp].copy()
    merged_timestamp = pd.merge(protein_timestamp, acq_timestamp[['RID', 'MRI_acquired']], on='RID', how='left', suffixes=('', '_MRI'))
    merged_dfs.append(merged_timestamp)

merged_df = pd.concat(merged_dfs, ignore_index=True)

# 8. Save merged dataframe
merged_df.to_csv(output_path, index=False)

print(f"\nMerged DataFrame saved to: {output_path}")

Number of protein bl (t=0): 295
Number of protein bl with MRI bl data: 43
Number of protein m12 (t=1): 71
Number of protein m12 with MRI m12 data: 29
Number of protein m24 (t=2): 4
Number of protein m24 with MRI m24 data: 3
Number of protein m36 (t=3): 4
Number of protein m36 with MRI m36 data: 2
Number of protein m48 (t=4): 5
Number of protein m48 with MRI m48 data: 2
Number of protein m60 (t=5): 0
Number of protein m60 with MRI m60 data: 0

Merged DataFrame saved to: E:\ADNI\AD_CN\protein\Hu Lab\protein_mri_merged.csv


# Protein: Fagan Lab

In [26]:
import pandas as pd

# File paths
protein = r"E:\ADNI\AD_CN\protein\Fagan Lab\FAGANLAB_07_15_2015_23Jun2025.csv"
img_acquisitions_info = r"E:\ADNI\AD_CN\AD,CN_Acquisitions_25Jun2025.csv"
output_path = r"E:\ADNI\AD_CN\protein\Fagan Lab\protein_mri_merged.csv"

# 1. Load protein data
protein_df = pd.read_csv(protein)

# 2. Load MRI acquisition info
acq_df = pd.read_csv(img_acquisitions_info)

# 3. Extract numeric RID from subject_id for matching
acq_df['RID'] = acq_df['subject_id'].str.split('_').str[-1].astype(int)

# 4. Define timestamps
timestamps = ['bl', 'm06', 'm12', 'm24', 'm36', 'm48', 'm60']

# 5. Initialize summary statistics
summary_stats = {}

# 6. Process each timestamp
for timestamp in timestamps:
    # Filter protein data for current timestamp
    protein_timestamp = protein_df[protein_df['VISCODE'].str.lower() == timestamp].copy()
    
    # Filter MRI acquisition info for current timestamp
    acq_timestamp = acq_df[acq_df['acquisition_visit'].str.lower() == timestamp].copy()
    
    # Count unique RIDs for protein data
    n_protein = protein_timestamp['RID'].nunique()
    
    # Merge protein and MRI info
    merged_timestamp = pd.merge(protein_timestamp, acq_timestamp[['RID', 'MRI_acquired']], on='RID', how='left', suffixes=('', '_MRI'))
    
    # Count protein data with MRI
    n_protein_with_mri = merged_timestamp['MRI_acquired'].eq(1).sum()
    
    # Store summary statistics
    summary_stats[timestamp] = {
        'n_protein': n_protein,
        'n_protein_with_mri': n_protein_with_mri
    }
    
    # Print summary for current timestamp
    print(f"Number of protein {timestamp} (t={timestamps.index(timestamp)}): {n_protein}")
    print(f"Number of protein {timestamp} with MRI {timestamp} data: {n_protein_with_mri}")

# 7. Combine all merged dataframes
merged_dfs = []
for timestamp in timestamps:
    protein_timestamp = protein_df[protein_df['VISCODE'].str.lower() == timestamp].copy()
    acq_timestamp = acq_df[acq_df['acquisition_visit'].str.lower() == timestamp].copy()
    merged_timestamp = pd.merge(protein_timestamp, acq_timestamp[['RID', 'MRI_acquired']], on='RID', how='left', suffixes=('', '_MRI'))
    merged_dfs.append(merged_timestamp)

merged_df = pd.concat(merged_dfs, ignore_index=True)

# 8. Save merged dataframe
merged_df.to_csv(output_path, index=False)

print(f"\nMerged DataFrame saved to: {output_path}")

Number of protein bl (t=0): 147
Number of protein bl with MRI bl data: 19
Number of protein m06 (t=1): 1
Number of protein m06 with MRI m06 data: 2
Number of protein m12 (t=2): 147
Number of protein m12 with MRI m12 data: 79
Number of protein m24 (t=3): 91
Number of protein m24 with MRI m24 data: 45
Number of protein m36 (t=4): 74
Number of protein m36 with MRI m36 data: 34
Number of protein m48 (t=5): 60
Number of protein m48 with MRI m48 data: 27
Number of protein m60 (t=6): 18
Number of protein m60 with MRI m60 data: 10

Merged DataFrame saved to: E:\ADNI\AD_CN\protein\Fagan Lab\protein_mri_merged.csv


# Protein : Cruchaga Lab

In [27]:
import pandas as pd

# File paths
protein = r"E:\ADNI\AD_CN\protein\CruchagaLab\CruchagaLab_CSF_SOMAscan7k_Protein_matrix_postQC_20230620.csv"
img_acquisitions_info = r"E:\ADNI\AD_CN\AD,CN_Acquisitions_25Jun2025.csv"
output_path = r"E:\ADNI\AD_CN\protein\CruchagaLab\protein_mri_merged.csv"

# 1. Load protein data
protein_df = pd.read_csv(protein)

# 2. Remove rows with NA in VISCODE2
protein_df = protein_df.dropna(subset=['VISCODE2'])

# 3. Load MRI acquisition info
acq_df = pd.read_csv(img_acquisitions_info)

# 4. Extract numeric RID from subject_id for matching
acq_df['RID'] = acq_df['subject_id'].str.split('_').str[-1].astype(int)

# 5. Define timestamps
timestamps = ['bl', 'm06', 'm12', 'm24', 'm36', 'm48', 'm60']

# 6. Initialize summary statistics
summary_stats = {}

# 7. Process each timestamp
for timestamp in timestamps:
    # Filter protein data for current timestamp
    protein_timestamp = protein_df[protein_df['VISCODE2'].str.lower() == timestamp].copy()
    
    # Filter MRI acquisition info for current timestamp
    acq_timestamp = acq_df[acq_df['acquisition_visit'].str.lower() == timestamp].copy()
    
    # Count unique RIDs for protein data
    n_protein = protein_timestamp['RID'].nunique()
    
    # Merge protein and MRI info
    merged_timestamp = pd.merge(protein_timestamp, acq_timestamp[['RID', 'MRI_acquired']], on='RID', how='left', suffixes=('', '_MRI'))
    
    # Count protein data with MRI
    n_protein_with_mri = merged_timestamp['MRI_acquired'].eq(1).sum()
    
    # Store summary statistics
    summary_stats[timestamp] = {
        'n_protein': n_protein,
        'n_protein_with_mri': n_protein_with_mri
    }
    
    # Print summary for current timestamp
    print(f"Number of protein {timestamp} (t={timestamps.index(timestamp)}): {n_protein}")
    print(f"Number of protein {timestamp} with MRI {timestamp} data: {n_protein_with_mri}")

# 8. Combine all merged dataframes
merged_dfs = []
for timestamp in timestamps:
    protein_timestamp = protein_df[protein_df['VISCODE2'].str.lower() == timestamp].copy()
    acq_timestamp = acq_df[acq_df['acquisition_visit'].str.lower() == timestamp].copy()
    merged_timestamp = pd.merge(protein_timestamp, acq_timestamp[['RID', 'MRI_acquired']], on='RID', how='left', suffixes=('', '_MRI'))
    merged_dfs.append(merged_timestamp)

merged_df = pd.concat(merged_dfs, ignore_index=True)

# 9. Save merged dataframe
merged_df.to_csv(output_path, index=False)

print(f"\nMerged DataFrame saved to: {output_path}")

Number of protein bl (t=0): 708
Number of protein bl with MRI bl data: 35
Number of protein m06 (t=1): 0
Number of protein m06 with MRI m06 data: 0
Number of protein m12 (t=2): 0
Number of protein m12 with MRI m12 data: 0
Number of protein m24 (t=3): 5
Number of protein m24 with MRI m24 data: 1
Number of protein m36 (t=4): 5
Number of protein m36 with MRI m36 data: 2
Number of protein m48 (t=5): 4
Number of protein m48 with MRI m48 data: 1
Number of protein m60 (t=6): 5
Number of protein m60 with MRI m60 data: 1

Merged DataFrame saved to: E:\ADNI\AD_CN\protein\CruchagaLab\protein_mri_merged.csv


# Proteomics: FNIH

In [28]:
import pandas as pd

# File paths
protein = r"E:\ADNI\AD_CN\proteomics\FNIH\csfproteomics\adni_csfproteomics.csv"
img_acquisitions_info = r"E:\ADNI\AD_CN\AD,CN_Acquisitions_25Jun2025.csv"
output_path = r"E:\ADNI\AD_CN\proteomics\FNIH\csfproteomics\protein_mri_merged.csv"

# 1. Load protein data
protein_df = pd.read_csv(protein)

# 2. Remove rows with NA in VISCODE
protein_df = protein_df.dropna(subset=['VISCODE'])

# 3. Load MRI acquisition info
acq_df = pd.read_csv(img_acquisitions_info)

# 4. Extract numeric RID from subject_id for matching
acq_df['RID'] = acq_df['subject_id'].str.split('_').str[-1].astype(int)

# 5. Define timestamps
timestamps = ['bl', 'm06', 'm12', 'm24', 'm36', 'm48', 'm60', 'm72', 'm84', 'm96', 'm108', 'm120']

# 6. Initialize summary statistics
summary_stats = {}

# 7. Process each timestamp
for timestamp in timestamps:
    # Filter protein data for current timestamp
    protein_timestamp = protein_df[protein_df['VISCODE'].str.lower() == timestamp].copy()
    
    # Filter MRI acquisition info for current timestamp
    acq_timestamp = acq_df[acq_df['acquisition_visit'].str.lower() == timestamp].copy()
    
    # Count unique RIDs for protein data
    n_protein = protein_timestamp['RID'].nunique()
    
    # Merge protein and MRI info
    merged_timestamp = pd.merge(protein_timestamp, acq_timestamp[['RID', 'MRI_acquired']], on='RID', how='left', suffixes=('', '_MRI'))
    
    # Count protein data with MRI
    n_protein_with_mri = merged_timestamp['MRI_acquired'].eq(1).sum()
    
    # Store summary statistics
    summary_stats[timestamp] = {
        'n_protein': n_protein,
        'n_protein_with_mri': n_protein_with_mri
    }
    
    # Print summary for current timestamp
    print(f"Number of protein {timestamp} (t={timestamps.index(timestamp)}): {n_protein}")
    print(f"Number of protein {timestamp} with MRI {timestamp} data: {n_protein_with_mri}")

# 8. Combine all merged dataframes
merged_dfs = []
for timestamp in timestamps:
    protein_timestamp = protein_df[protein_df['VISCODE'].str.lower() == timestamp].copy()
    acq_timestamp = acq_df[acq_df['acquisition_visit'].str.lower() == timestamp].copy()
    merged_timestamp = pd.merge(protein_timestamp, acq_timestamp[['RID', 'MRI_acquired']], on='RID', how='left', suffixes=('', '_MRI'))
    merged_dfs.append(merged_timestamp)

merged_df = pd.concat(merged_dfs, ignore_index=True)

# 9. Save merged dataframe
merged_df.to_csv(output_path, index=False)

print(f"\nMerged DataFrame saved to: {output_path}")

Number of protein bl (t=0): 111
Number of protein bl with MRI bl data: 15
Number of protein m06 (t=1): 0
Number of protein m06 with MRI m06 data: 0
Number of protein m12 (t=2): 83
Number of protein m12 with MRI m12 data: 51
Number of protein m24 (t=3): 37
Number of protein m24 with MRI m24 data: 19
Number of protein m36 (t=4): 47
Number of protein m36 with MRI m36 data: 24
Number of protein m48 (t=5): 49
Number of protein m48 with MRI m48 data: 25
Number of protein m60 (t=6): 16
Number of protein m60 with MRI m60 data: 11
Number of protein m72 (t=7): 0
Number of protein m72 with MRI m72 data: 0
Number of protein m84 (t=8): 0
Number of protein m84 with MRI m84 data: 0
Number of protein m96 (t=9): 0
Number of protein m96 with MRI m96 data: 0
Number of protein m108 (t=10): 0
Number of protein m108 with MRI m108 data: 0
Number of protein m120 (t=11): 0
Number of protein m120 with MRI m120 data: 0

Merged DataFrame saved to: E:\ADNI\AD_CN\proteomics\FNIH\csfproteomics\protein_mri_merged.csv

# Proteomics: Biomarkers Consortium Project - Plasma

In [23]:
import pandas as pd

# File paths
protein_qc = r"E:\ADNI\AD_CN\proteomics\Biomarkers Consortium Plasma Proteomics Plasma\adni_plasma_qc_multiplex_11Nov2010.csv"
img_acquisitions_info = r"E:\ADNI\AD_CN\AD,CN_Acquisitions_25Jun2025.csv"
output_path = r"E:\ADNI\AD_CN\proteomics\Biomarkers Consortium Plasma Proteomics Plasma\protein_mri_merged.csv"

# 1. Load protein QC data
protein_df = pd.read_csv(protein_qc)

# 2. Count unique RIDs for 'bl' and 'm12'
n_protein_bl = protein_df[protein_df['Visit_Code'].str.lower() == 'bl']['RID'].nunique()
print(f"Number of protein baseline (t=0): {n_protein_bl}")

n_protein_m12 = protein_df[protein_df['Visit_Code'].str.lower() == 'm12']['RID'].nunique()
print(f"Number of protein m12: {n_protein_m12}")


# 3. Load MRI acquisition info
acq_df = pd.read_csv(img_acquisitions_info)

# 4. Prepare MRI info for 'bl' and 'm12'
acq_bl = acq_df[acq_df['acquisition_visit'].str.lower() == 'bl'].copy()
acq_m12 = acq_df[acq_df['acquisition_visit'].str.lower() == 'm12'].copy()

# Extract numeric RID from subject_id for matching
acq_bl['RID'] = acq_bl['subject_id'].str.split('_').str[-1].astype(int)
acq_m12['RID'] = acq_m12['subject_id'].str.split('_').str[-1].astype(int)

# 5. Merge protein and MRI info for 'bl' and 'm12'
protein_bl = protein_df[protein_df['Visit_Code'].str.lower() == 'bl'].copy()
protein_m12 = protein_df[protein_df['Visit_Code'].str.lower() == 'm12'].copy()

merged_bl = pd.merge(protein_bl, acq_bl[['RID', 'MRI_acquired']], on='RID', how='left', suffixes=('', '_MRI'))
merged_m12 = pd.merge(protein_m12, acq_m12[['RID', 'MRI_acquired']], on='RID', how='left', suffixes=('', '_MRI'))

# 6. Summary statistics
n_protein_bl_with_mri = merged_bl['MRI_acquired'].eq(1).sum()
n_protein_m12_with_mri = merged_m12['MRI_acquired'].eq(1).sum()

print(f"Number of protein baseline (t=0): {n_protein_bl}")
print(f"Number of protein baseline with MRI baseline data: {n_protein_bl_with_mri}")
print(f"Number of protein m12: {n_protein_m12}")
print(f"Number of protein m12 with MRI m12 data: {n_protein_m12_with_mri}")

# 7. Combine merged_bl and merged_m12 for output
merged_df = pd.concat([merged_bl, merged_m12], ignore_index=True)

# 8. Save merged dataframe
merged_df.to_csv(output_path, index=False)

Number of protein baseline (t=0): 566
Number of protein m12: 496
Number of protein baseline (t=0): 566
Number of protein baseline with MRI baseline data: 40
Number of protein m12: 496
Number of protein m12 with MRI m12 data: 162


# Proteomics: Biomarkers Consortium Project - MRM

In [1]:
import pandas as pd

# File paths
protein = r"E:\ADNI\AD_CN\proteomics\Biomarkers Consortium Plasma Proteomics MRM\CSFMRM_23Jun2025.csv"
img_acquisitions_info = r"E:\ADNI\AD_CN\AD,CN_Acquisitions_25Jun2025.csv"
output_path = r"E:\ADNI\AD_CN\proteomics\Biomarkers Consortium Plasma Proteomics MRM\protein_mri_merged.csv"

# 1. Load protein data
protein_df = pd.read_csv(protein)

# 2. Count unique RIDs for baseline (t=0)
n_protein_bl = protein_df[protein_df['VISCODE'].str.lower() == 'bl']['RID'].nunique()

# 3. Load MRI acquisition info
acq_df = pd.read_csv(img_acquisitions_info)

# 4. Prepare MRI info for baseline (t=0)
acq_bl = acq_df[acq_df['acquisition_visit'].str.lower() == 'bl'].copy()

# Extract numeric RID from subject_id for matching
acq_bl['RID'] = acq_bl['subject_id'].str.split('_').str[-1].astype(int)

# 5. Merge protein and MRI info for baseline (t=0)
protein_bl = protein_df[protein_df['VISCODE'].str.lower() == 'bl'].copy()

merged_bl = pd.merge(protein_bl, acq_bl[['RID', 'MRI_acquired']], on='RID', how='left', suffixes=('', '_MRI'))

# 6. Summary statistics
n_protein_bl_with_mri = merged_bl['MRI_acquired'].eq(1).sum()

print(f"Number of protein baseline (t=0): {n_protein_bl}")
print(f"Number of protein baseline with MRI baseline data: {n_protein_bl_with_mri}")

# 7. Save merged dataframe
merged_bl.to_csv(output_path, index=False)

Number of protein baseline (t=0): 287
Number of protein baseline with MRI baseline data: 45


In [6]:
merged_bl.head()

,RID,VISCODE,A1AT_AVLTIDEK,A1AT_LSITGTYDLK,A1AT_SVLGQLGITK,A1BG_NGVAQEPVHLDSPAIK,A1BG_SGLSTGWTQLSK,A2GL_DLLLPQPDLR,A2GL_VAAGAFQGLR,A4_LVFFAEDVGSNK,...,VASN_NLHDLDVSDNQLER,VASN_SLTLGIEPVSPTSLR,VASN_YLQGSSVQLR,VGF_AYQGVAAPFPK,VGF_NSEPQDEGELFQGVDPR,VGF_THLGEALAPLSK,VTDB_EFSHLGK,VTDB_HLSLLTTLSNR,VTDB_VPTADLEDVLPLAEDITNILSK,MRI_acquired
0,3,bl,10.09220,7.74523,10.17220,16.6720,21.4594,26.9142,24.2002,17.5886,...,14.8820,12.4679,15.9046,16.7334,18.3733,15.7735,10.82520,21.7876,17.6057,NaN
1,4,bl,10.94970,7.70817,9.57396,14.9759,19.5672,25.3480,22.5761,17.9311,...,14.5208,11.9206,15.5596,17.1372,19.1806,15.8797,9.33075,19.0783,15.5337,NaN
2,5,bl,9.98930,6.60892,9.68137,15.2048,19.7987,25.7420,22.8861,18.0174,...,14.3662,12.1692,15.7139,17.9062,19.4055,16.8510,9.17437,19.1255,15.0267,NaN
3,8,bl,9.81022,7.65311,10.21570,15.6723,20.3279,25.4989,22.4874,18.6170,...,15.4665,13.1469,16.5607,19.6158,21.2765,18.4325,9.86449,20.2941,16.1640,NaN
4,10,bl,8.92968,7.58228,9.95119,15.7266,20.1880,26.0936,23.2094,18.1481,...,14.9748,12.4011,15.8251,17.6525,19.5023,16.5219,9.27747,19.8080,15.8384,NaN


In [ ]:

# Load class label data
labels_path = r"E:\ADNI\AD_CN\AD,CN_First_Visit_25Jun2025.csv"
labels_df = pd.read_csv(labels_path)

# Extract RID from subject_id
labels_df['RID'] = labels_df['subject_id'].str.split('_').str[-1].astype(int)
labels_df.head()

# Keep only RID and research group
labels_clean = labels_df[['RID', 'research_group']]

# Merge with the previously merged protein-MRI data
merged_with_labels = pd.merge(merged_bl, labels_clean, on='RID', how='left')

# Count number of subjects per research group in the 45 paired data
class_counts = merged_with_labels[merged_with_labels['MRI_acquired'] == 1]['research_group'].value_counts()

class_counts.to_dict()

{'CN': 25, 'AD': 20}

In [9]:
import pandas as pd

# Load class label data
labels_path = r"E:\ADNI\AD_CN\AD,CN_First_Visit_25Jun2025.csv"
labels_df = pd.read_csv(labels_path)

# Extract RID from subject_id
labels_df['RID'] = labels_df['subject_id'].str.split('_').str[-1].astype(int)

# Merge subject labels (including age and group) with protein-MRI merged data
labels_clean = labels_df[['RID', 'research_group', 'subject_age']]
merged_with_labels = pd.merge(merged_bl, labels_clean, on='RID', how='left')

# Filter for paired protein-MRI data
paired_data = merged_with_labels[merged_with_labels['MRI_acquired'] == 1]
paired_data.head()
# Count of subjects per group
class_counts = paired_data['research_group'].value_counts()

# Average age per group
avg_age_by_group = paired_data.groupby('research_group')['subject_age'].mean().round(2)

# Combine both into a summary dictionary
summary_stats = {
    'class_counts': class_counts.to_dict(),
    'average_age_by_group': avg_age_by_group.to_dict()
}

summary_stats


{'class_counts': {'CN': 25, 'AD': 20},
 'average_age_by_group': {'AD': 70.42, 'CN': 75.92}}

# Proteomics: Biomarkers Consortium Project - Plasma

In [ ]:
import pandas as pd

# File paths
protein_qc = r"E:\ADNI\AD_CN\proteomics\Biomarkers Consortium Plasma Proteomics Plasma\adni_plasma_qc_multiplex_11Nov2010.csv"
img_acquisitions_info = r"E:\ADNI\AD_CN\AD,CN_Acquisitions_25Jun2025.csv"
output_path = r"E:\ADNI\AD_CN\proteomics\Biomarkers Consortium Plasma Proteomics Plasma\protein_mri_merged.csv"

# 1. Load protein QC data
protein_df = pd.read_csv(protein_qc)

# 2. Count unique RIDs for 'bl' and 'm12'
n_protein_bl = protein_df[protein_df['Visit_Code'].str.lower() == 'bl']['RID'].nunique()
print(f"Number of protein baseline (t=0): {n_protein_bl}")

n_protein_m12 = protein_df[protein_df['Visit_Code'].str.lower() == 'm12']['RID'].nunique()
print(f"Number of protein m12: {n_protein_m12}")


# 3. Load MRI acquisition info
acq_df = pd.read_csv(img_acquisitions_info)

# 4. Prepare MRI info for 'bl' and 'm12'
acq_bl = acq_df[acq_df['acquisition_visit'].str.lower() == 'bl'].copy()
acq_m12 = acq_df[acq_df['acquisition_visit'].str.lower() == 'm12'].copy()

# Extract numeric RID from subject_id for matching
acq_bl['RID'] = acq_bl['subject_id'].str.split('_').str[-1].astype(int)
acq_m12['RID'] = acq_m12['subject_id'].str.split('_').str[-1].astype(int)

# 5. Merge protein and MRI info for 'bl' and 'm12'
protein_bl = protein_df[protein_df['Visit_Code'].str.lower() == 'bl'].copy()
protein_m12 = protein_df[protein_df['Visit_Code'].str.lower() == 'm12'].copy()

merged_bl = pd.merge(protein_bl, acq_bl[['RID', 'MRI_acquired']], on='RID', how='left', suffixes=('', '_MRI'))
merged_m12 = pd.merge(protein_m12, acq_m12[['RID', 'MRI_acquired']], on='RID', how='left', suffixes=('', '_MRI'))

# 6. Summary statistics
n_protein_bl_with_mri = merged_bl['MRI_acquired'].eq(1).sum()
n_protein_m12_with_mri = merged_m12['MRI_acquired'].eq(1).sum()

print(f"Number of protein baseline (t=0): {n_protein_bl}")
print(f"Number of protein baseline with MRI baseline data: {n_protein_bl_with_mri}")
print(f"Number of protein m12: {n_protein_m12}")
print(f"Number of protein m12 with MRI m12 data: {n_protein_m12_with_mri}")

# 7. Combine merged_bl and merged_m12 for output
merged_df = pd.concat([merged_bl, merged_m12], ignore_index=True)

# 8. Save merged dataframe
merged_df.to_csv(output_path, index=False)

Number of protein baseline (t=0): 566
Number of protein m12: 496
Number of protein baseline (t=0): 566
Number of protein baseline with MRI baseline data: 40
Number of protein m12: 496
Number of protein m12 with MRI m12 data: 162


# Proteomics: Biomarkers Consortium Project - CSF

In [11]:
import pandas as pd

# File paths
protein = r"E:\ADNI\AD_CN\proteomics\Biomarkers Consortium Plasma Proteomics CSF\Biomarkers Consortium ADNI CSF QC Multiplex data.csv"
img_acquisitions_info = r"E:\ADNI\AD_CN\AD,CN_Acquisitions_25Jun2025.csv"
output_path = r"E:\ADNI\AD_CN\proteomics\Biomarkers Consortium Plasma Proteomics CSF\protein_mri_merged.csv"

# 1. Load protein data
protein_df = pd.read_csv(protein)

# 2. Remove rows with 'NA' in visit_code
protein_df = protein_df[protein_df['visit_code'].str.lower() != 'na']

# 3. Load MRI acquisition info
acq_df = pd.read_csv(img_acquisitions_info)

# 4. Extract numeric rid from subject_id for matching
acq_df['rid'] = acq_df['subject_id'].str.split('_').str[-1].astype(int)

# 5. Get unique timestamps from protein data
unique_timestamps = protein_df['visit_code'].str.lower().unique()

# 6. Initialize summary statistics
summary_stats = {}

# 7. Process each timestamp
for timestamp in unique_timestamps:
    # Filter protein data for current timestamp
    protein_timestamp = protein_df[protein_df['visit_code'].str.lower() == timestamp].copy()
    
    # Filter MRI acquisition info for current timestamp
    acq_timestamp = acq_df[acq_df['acquisition_visit'].str.lower() == timestamp].copy()
    
    # Count unique rids for protein data
    n_protein = protein_timestamp['rid'].nunique()
    
    # Merge protein and MRI info
    merged_timestamp = pd.merge(protein_timestamp, acq_timestamp[['rid', 'MRI_acquired']], on='rid', how='left', suffixes=('', '_MRI'))
    
    # Count protein data with MRI
    n_protein_with_mri = merged_timestamp['MRI_acquired'].eq(1).sum()
    
    # Store summary statistics
    summary_stats[timestamp] = {
        'n_protein': n_protein,
        'n_protein_with_mri': n_protein_with_mri
    }
    
    # Print summary for current timestamp
    print(f"Number of protein {timestamp} (t={list(unique_timestamps).index(timestamp)}): {n_protein}")
    print(f"Number of protein {timestamp} with MRI {timestamp} data: {n_protein_with_mri}")

# 8. Combine all merged dataframes
merged_dfs = []
for timestamp in unique_timestamps:
    protein_timestamp = protein_df[protein_df['visit_code'].str.lower() == timestamp].copy()
    acq_timestamp = acq_df[acq_df['acquisition_visit'].str.lower() == timestamp].copy()
    merged_timestamp = pd.merge(protein_timestamp, acq_timestamp[['rid', 'MRI_acquired']], on='rid', how='left', suffixes=('', '_MRI'))
    merged_dfs.append(merged_timestamp)

merged_df = pd.concat(merged_dfs, ignore_index=True)

# 9. Save merged dataframe
merged_df.to_csv(output_path, index=False)

print(f"\nMerged DataFrame saved to: {output_path}")

Number of protein bl (t=0): 311
Number of protein bl with MRI bl data: 46

Merged DataFrame saved to: E:\ADNI\AD_CN\proteomics\Biomarkers Consortium Plasma Proteomics CSF\protein_mri_merged.csv


In [13]:
# File paths
labels_path = r"E:\ADNI\AD_CN\AD,CN_First_Visit_25Jun2025.csv"

# Load class label data
labels_df = pd.read_csv(labels_path)

# Extract RID from subject_id
labels_df['rid'] = labels_df['subject_id'].str.split('_').str[-1].astype(int)

# Merge subject labels (including age and group) with the merged protein-MRI data
labels_clean = labels_df[['rid', 'research_group', 'subject_age']]
merged_with_labels = pd.merge(merged_df, labels_clean, on='rid', how='left')

# Filter for paired protein-MRI data
paired_data = merged_with_labels[merged_with_labels['MRI_acquired'] == 1]

# Count of subjects per group
class_counts = paired_data['research_group'].value_counts()

# Average age per group
avg_age_by_group = paired_data.groupby('research_group')['subject_age'].mean().round(2)

# Combine both into a summary dictionary
summary_stats_extended = {
    'class_counts': class_counts.to_dict(),
    'average_age_by_group': avg_age_by_group.to_dict()
}

summary_stats_extended


{'class_counts': {'CN': 26, 'AD': 20},
 'average_age_by_group': {'AD': 70.42, 'CN': 75.72}}

In [20]:
paired_df = paired_df.loc[:, ~(paired_df == '.').all()]
paired_df

,rid,sampid,AXL Receptor Tyrosine Kinase (AXL) (ng/mL),Adiponectin (ug/mL),Agouti-Related Protein (AGRP) (pg/mL),Alpha-1-Antitrypsin (AAT) (mg/mL),Alpha-1-Microglobulin (A1Micro) (ug/ml),Alpha-2-Macroglobulin (A2Macro) (mg/mL),Angiopoietin-2 (ANG-2) (ng/mL),Angiotensin-Converting Enzyme (ACE) (ng/ml),...,Tissue Inhibitor of Metalloproteinases 1 (ng/mL),Transforming Growth Factor alpha (TGF-al (pg/mL),Trefoil Factor 3 (TFF3) (ug/ml),Tumor Necrosis Factor Receptor 2 (TNFR2) (ng/mL),Vascular Cell Adhesion Molecule-1 (VCAM- (ng/mL),Vascular Endothelial Growth Factor (VEGF (pg/mL),von Willebrand Factor (vWF) (ug/mL),id,visit_code,MRI_acquired
10,29,JA801DCV-29,3.5,-2.292430,77,-2.387216,-1.619789,-2.420216,-0.130768,0.113943,...,1.544068,0.838849,-2.214670,-0.214670,1.113943,2.597695,-1.657577,11,bl,1.0
11,31,FA801DV5-08,4.1,-2.376751,104,-2.346787,-1.283997,-2.356547,0.278754,0.505150,...,1.662758,0.934498,-1.744727,-0.075721,1.255273,2.777427,-1.292430,12,bl,1.0
23,61,JA801D0P-08,4.6,-2.552842,60,-2.494850,-1.408935,-2.420216,-0.050610,0.322219,...,1.447158,0.934498,-1.537602,-0.229148,1.079181,2.613842,-1.494850,24,bl,1.0
30,86,BA8034Y1-28,2.7,-2.124939,55,-2.387216,-1.356547,-2.244125,-0.207608,0.041393,...,1.732394,0.785330,-1.958607,-0.091515,1.230449,2.632457,-1.568636,31,bl,1.0
48,139,JA801KPJ-10,3.8,-2.420216,60,-2.508638,-1.585027,-2.602060,0.113943,0.079181,...,1.518514,0.740363,-1.920819,-0.301030,0.934498,2.629410,-1.494850,49,bl,1.0
49,139,JA801KPJ-11,3.1,-2.376751,60,-2.585027,-1.552842,-2.376751,0.176091,0.146128,...,1.477121,1.146128,-1.920819,-0.187087,0.977724,2.658011,-1.481486,50,bl,1.0
76,260,HA8015T8-08,4.1,-2.619789,37,-2.292430,-1.455932,-2.585027,-0.142668,0.000000,...,1.414973,1.041393,-1.886057,-0.275724,1.041393,2.743510,-1.387216,77,bl,1.0
86,300,KA801ZWN-29,3.0,-2.251812,55,-2.387216,-1.585027,-2.376751,-0.244125,0.079181,...,1.770852,0.653213,-1.823909,-0.075721,1.230449,2.644439,-1.585027,87,bl,1.0
108,403,DA80194Y-24,6.0,-2.200659,33,-2.207608,-1.075721,-2.408935,-0.070581,0.380211,...,1.633468,1.079181,-1.602060,-0.113509,1.176091,2.812245,-1.420216,109,bl,1.0
110,413,AA8018LB-06,2.3,-2.154902,37,-2.292430,-1.508638,-2.552842,-0.251812,0.204120,...,1.462398,1.079181,-1.920819,-0.455932,0.919078,2.481443,-1.638272,111,bl,1.0


# Data Harmonization Analysis

In [ ]:
import pandas as pd

# File paths for merged datasets
BC_CSF = r"E:\ADNI\AD_CN\proteomics\Biomarkers Consortium Plasma Proteomics CSF\BC_CSF_protein_mri_merged.csv"
FNIH = r"E:\ADNI\AD_CN\proteomics\FNIH\csfproteomics\FNIH_protein_mri_merged.csv"
Hu_lab = r"E:\ADNI\AD_CN\protein\Hu Lab\Hu_protein_mri_merged.csv"
Fagan_lab = r"E:\ADNI\AD_CN\protein\Fagan Lab\Fagan_protein_mri_merged.csv"

# Load each dataset
bc_csf_df = pd.read_csv(BC_CSF)
fnih_df = pd.read_csv(FNIH)
hu_df = pd.read_csv(Hu_lab)
fagan_df = pd.read_csv(Fagan_lab)

# Filter for MRI_acquired == 1 (handle both int and str)
def filter_mri_acquired(df):
    return df[df['MRI_acquired'] == 1]

bc_csf_mri = filter_mri_acquired(bc_csf_df)
fnih_mri = filter_mri_acquired(fnih_df)
hu_mri = filter_mri_acquired(hu_df)
fagan_mri = filter_mri_acquired(fagan_df)

# Save to Excel with 4 sheets
output_excel = r"E:\ADNI\AD_CN\BC_FNIH_Hu_Fagan.xlsx"
with pd.ExcelWriter(output_excel) as writer:
    bc_csf_mri.to_excel(writer, sheet_name='BC_CSF', index=False)
    fnih_mri.to_excel(writer, sheet_name='FNIH', index=False)
    hu_mri.to_excel(writer, sheet_name='Hu_lab', index=False)
    fagan_mri.to_excel(writer, sheet_name='Fagan_lab', index=False)

print(f"Filtered datasets saved to {output_excel}")

In [11]:
import pandas as pd

# Read the Excel file
excel_path = r"E:\ADNI\AD_CN\BC_FNIH_Hu_Fagan.xlsx"
sheets = pd.read_excel(excel_path, sheet_name=None)

# Extract dataframes
bc_csf = sheets['BC_CSF']
fnih = sheets['FNIH']
hu_lab = sheets['Hu_lab']
fagan_lab = sheets['Fagan_lab']

# For FNIH, Hu_lab, Fagan_lab, filter for baseline (VISCODE == 'bl', case-insensitive)
def filter_bl(df):
    if 'VISCODE' in df.columns:
        return df[df['VISCODE'].str.lower() == 'bl']
    return df

fnih_bl = filter_bl(fnih)
hu_lab_bl = filter_bl(hu_lab)
fagan_lab_bl = filter_bl(fagan_lab)

# Get all unique RIDs across all datasets at baseline
all_rids = set(bc_csf['RID']) | set(fnih_bl['RID']) | set(hu_lab_bl['RID']) | set(fagan_lab_bl['RID'])

all_rids
# Build the presence table
tick = '\u2713'  # tick mark
cross = '\u2717'  # cross mark
data = []
for rid in sorted(all_rids):
    row = {
        'RID': rid,
        'BC_CSF': tick if rid in set(bc_csf['RID']) else cross,
        'FNIH': tick if rid in set(fnih_bl['RID']) else cross,
        'Hu_lab': tick if rid in set(hu_lab_bl['RID']) else cross,
        'Fagan_lab': tick if rid in set(fagan_lab_bl['RID']) else cross,
    }
    data.append(row)

presence_df = pd.DataFrame(data)

# Show the first few rows of the table
print(presence_df)

# # Optionally, save the table to Excel
# presence_df.to_excel(r"E:\ADNI\AD_CN\BC_FNIH_Hu_Fagan_overlap_table.xlsx", index=False)


     RID BC_CSF FNIH Hu_lab Fagan_lab
0     29      ✓    ✗      ✓         ✗
1     31      ✓    ✓      ✓         ✓
2     61      ✓    ✓      ✓         ✓
3     86      ✓    ✗      ✗         ✗
4    139      ✓    ✓      ✓         ✓
5    194      ✗    ✗      ✓         ✗
6    196      ✗    ✗      ✓         ✗
7    260      ✓    ✓      ✓         ✓
8    299      ✗    ✗      ✓         ✗
9    300      ✓    ✗      ✓         ✗
10   403      ✓    ✓      ✓         ✓
11   404      ✗    ✗      ✓         ✗
12   413      ✓    ✗      ✓         ✗
13   426      ✓    ✗      ✓         ✗
14   433      ✓    ✗      ✓         ✗
15   441      ✓    ✗      ✓         ✗
16   457      ✓    ✗      ✓         ✗
17   479      ✓    ✓      ✓         ✓
18   488      ✓    ✗      ✗         ✗
19   525      ✓    ✗      ✓         ✗
20   559      ✓    ✓      ✓         ✓
21   602      ✓    ✗      ✓         ✗
22   605      ✓    ✗      ✓         ✗
23   606      ✓    ✗      ✓         ✗
24   691      ✓    ✗      ✓         ✗
25   724    

In [13]:
# Calculate how many rows have all ticks (i.e., all datasets present for that RID)
all_tick_cols = ['BC_CSF', 'FNIH', 'Hu_lab', 'Fagan_lab']
all_ticks_count = (presence_df[all_tick_cols] == tick).all(axis=1).sum()
print(f"Number of RIDs present in all datasets (all ticks): {all_ticks_count}")

# Calculate, for each pair of datasets, how many RIDs have ticks in both
from itertools import combinations

print("\nNumber of RIDs present in each pair of datasets (both ticks):")
for col1, col2 in combinations(all_tick_cols, 2):
    both_ticks = ((presence_df[col1] == tick) & (presence_df[col2] == tick)).sum()
    print(f"{col1} & {col2}: {both_ticks}")


Number of RIDs present in all datasets (all ticks): 11

Number of RIDs present in each pair of datasets (both ticks):
BC_CSF & FNIH: 15
BC_CSF & Hu_lab: 30
BC_CSF & Fagan_lab: 19
FNIH & Hu_lab: 11
FNIH & Fagan_lab: 15
Hu_lab & Fagan_lab: 13


In [19]:
## Combine with CN or AD labels

# Load subject info
subject_info = pd.read_csv(r"E:\ADNI\AD_CN\AD,CN_My_Table_25Jun2025.csv")

# Only keep baseline rows
subject_info_bl = subject_info[subject_info['visit'].str.lower() == 'bl'].copy()

# Helper to merge subject info into a dataset by subject_id
def merge_subject_info(df, rid_col='RID'):
    # Ensure RID is int for matching
    df = df.copy()
    df[rid_col] = df[rid_col].astype(int)
    # Merge on RID <-> subject_id's numeric part
    merged = pd.merge(
        df,
        subject_info_bl,
        left_on=rid_col,
        right_on=subject_info_bl['subject_id'].str.split('_').str[-1].astype(int),
        how='left',
        suffixes=('', '_subject')
    )
    # Keep only the requested columns from subject_info
    keep_cols = ['subject_id', 'visit', 'PROTEIN', 'research_group', 'subject_date', 'subject_age']
    for col in keep_cols:
        if col not in merged.columns:
            merged[col] = None
    return merged

# Read the Excel file
excel_path = r"E:\ADNI\AD_CN\BC_FNIH_Hu_Fagan.xlsx"
sheets = pd.read_excel(excel_path, sheet_name=None)

# For FNIH, Hu_lab, Fagan_lab, filter for baseline (VISCODE == 'bl', case-insensitive)
def filter_bl(df):
    if 'VISCODE' in df.columns:
        return df[df['VISCODE'].str.lower() == 'bl']
    return df

# Merge and prepare new sheets
new_sheets = {}
for name in ['BC_CSF', 'FNIH', 'Hu_lab', 'Fagan_lab']:
    df = sheets[name]
    if name != 'BC_CSF':
        df = filter_bl(df)
    merged = merge_subject_info(df, rid_col='RID')
    new_sheets[name + '_with_labels'] = merged

# Save to a new Excel file (or overwrite)
output_excel = r"E:\ADNI\AD_CN\BC_FNIH_Hu_Fagan_with_labels.xlsx"
with pd.ExcelWriter(output_excel) as writer:
    for name, df in new_sheets.items():
        df.to_excel(writer, sheet_name=name, index=False)

print(f"New Excel file with subject info saved to {output_excel}")

New Excel file with subject info saved to E:\ADNI\AD_CN\BC_FNIH_Hu_Fagan_with_labels.xlsx
